In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, sqrt
from pyspark.sql.types import DoubleType
import math

spark = SparkSession.builder \
    .appName("Temperature Analysis") \
    .getOrCreate()


23/04/10 20:55:59 WARN Utils: Your hostname, Ritheshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.235 instead (on interface en0)
23/04/10 20:55:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 20:56:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Read a CSV file into a DataFrame
df_stations = spark.read.csv("stations.csv")

df_stations = df_stations.dropDuplicates()
df_stations = df_stations.dropna(subset=["_c2", "_c3"], how="all")

df_stations.show()

+------+----+-------+--------+
|   _c0| _c1|    _c2|     _c3|
+------+----+-------+--------+
|014190|null|+59.050|+006.117|
|020840|null|+68.050|+022.000|
|021670|null|+55.383|+012.817|
|022690|null|+63.183|+019.017|
|027880|null|+63.150|+027.317|
|029740|null|+60.317|+024.963|
|033020|null|+53.248|-004.535|
|041400|null|+63.850|-021.383|
|041950|null|+65.280|-014.000|
|062670|null|+52.900|+005.383|
|071540|null|+48.800|+002.500|
|072660|null|+47.850|+003.497|
|080550|null|+42.589|-005.656|
|082330|null|+40.933|-001.300|
|085440|null|+40.917|-008.633|
|106450|null|+49.900|+009.433|
|112980|null|+46.933|+015.867|
|128430|null|+47.433|+019.183|
|131680|null|+45.333|+019.850|
|153350|null|+45.062|+028.714|
+------+----+-------+--------+
only showing top 20 rows



In [10]:
df_stations.count()


28129

In [11]:
df_1986 = spark.read.csv("1986.csv")
df_1986 = df_1986.dropDuplicates()

df_1986 = df_1986.dropDuplicates()
df_1986 = df_1986.dropna(subset=["_c2", "_c3"], how="all")

df_1986.show()

+------+----+---+---+----+
|   _c0| _c1|_c2|_c3| _c4|
+------+----+---+---+----+
|010010|null| 03| 10|29.4|
|010010|null| 04| 05|34.9|
|010080|null| 01| 11| 0.1|
|010080|null| 09| 03|41.3|
|010080|null| 09| 05|41.9|
|010080|null| 09| 25|20.5|
|010100|null| 08| 04|48.4|
|010100|null| 08| 25|48.4|
|010100|null| 09| 16|42.0|
|010230|null| 01| 06|18.7|
|010230|null| 05| 26|52.3|
|010250|null| 05| 04|41.6|
|010250|null| 07| 15|54.9|
|010330|null| 06| 22|44.6|
|010350|null| 06| 02|50.0|
|010350|null| 07| 16|55.2|
|010520|null| 03| 16|41.2|
|010520|null| 05| 21|46.3|
|010530|null| 08| 02|67.8|
|010530|null| 08| 28|43.2|
+------+----+---+---+----+
only showing top 20 rows



In [21]:
temperatures_df = df_1986.join(df_stations, ["_c0", "_c1"], "left_outer")
temperatures_df.show()

+------+----+---+---+----+----+----+
|   _c0| _c1|_c2|_c3| _c4| _c2| _c3|
+------+----+---+---+----+----+----+
|010010|null| 03| 10|29.4|null|null|
|010010|null| 04| 05|34.9|null|null|
|010080|null| 01| 11| 0.1|null|null|
|010080|null| 09| 03|41.3|null|null|
|010080|null| 09| 05|41.9|null|null|
|010080|null| 09| 25|20.5|null|null|
|010100|null| 08| 04|48.4|null|null|
|010100|null| 08| 25|48.4|null|null|
|010100|null| 09| 16|42.0|null|null|
|010230|null| 01| 06|18.7|null|null|
|010230|null| 05| 26|52.3|null|null|
|010250|null| 05| 04|41.6|null|null|
|010250|null| 07| 15|54.9|null|null|
|010330|null| 06| 22|44.6|null|null|
|010350|null| 06| 02|50.0|null|null|
|010350|null| 07| 16|55.2|null|null|
|010520|null| 03| 16|41.2|null|null|
|010520|null| 05| 21|46.3|null|null|
|010530|null| 08| 02|67.8|null|null|
|010530|null| 08| 28|43.2|null|null|
+------+----+---+---+----+----+----+
only showing top 20 rows



In [17]:
def haversine_distance(lat1, lon1, lat2, lon2):
    
    R = 6371  # Earth's radius in km
    
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c



In [24]:
df_stations = df_stations.dropna(subset=["_c2", "_c3"])
temperatures_df = temperatures_df.filter((col("DATE") == "1986-01-28"))

AnalysisException: Column 'DATE' does not exist. Did you mean one of the following? [_c0, _c1, _c2, _c2, _c3, _c3, _c4];
'Filter ('DATE = 1986-01-28)
+- Project [_c0#321, _c1#322, _c2#323, _c3#324, _c4#325, _c2#246, _c3#247]
   +- Join LeftOuter, ((_c0#321 = _c0#244) AND (_c1#322 = _c1#245))
      :- Filter atleastnnonnulls(1, _c2#323, _c3#324)
      :  +- Deduplicate [_c3#324, _c0#321, _c4#325, _c1#322, _c2#323]
      :     +- Deduplicate [_c3#324, _c0#321, _c4#325, _c1#322, _c2#323]
      :        +- Relation [_c0#321,_c1#322,_c2#323,_c3#324,_c4#325] csv
      +- Filter atleastnnonnulls(1, _c2#246, _c3#247)
         +- Deduplicate [_c0#244, _c1#245, _c2#246, _c3#247]
            +- Relation [_c0#244,_c1#245,_c2#246,_c3#247] csv


In [ ]:
cape_canaveral_coordinates = (28.3922, -80.6077)

nearby_stations = stations_df.withColumn("distance", haversine_udf(col("LATITUDE"), col("LONGITUDE"), lit(cape_canaveral_coordinates[0]), lit(cape_canaveral_coordinates[1]))).filter(col("distance") <= 100)

temp_nearby_stations = temperatures_df.join(nearby_stations, "STATION")
temp_nearby_stations = temp_nearby_stations.withColumn("weight", 1 / col("distance"))
weighted_temperature_sum = temp_nearby_stations.agg({"weight": "sum"}).collect()[0][0]
temp_nearby_stations = temp_nearby_stations.withColumn("weighted_temperature", col("TEMPERATURE") * col("weight"))
weighted_temperature_at_cape = temp_nearby_stations.agg({"weighted_temperature": "sum"}).collect()[0][0] / weighted_temperature_sum

print(f"Estimated temperature at Cape Canaveral on January 28, 1986: {weighted_temperature_at_cape:.2f}°C")

In [ ]:
 

Rubric 

 

Clarity & Critical Thinking  


 

Reasoning & Support  



Creativity & Risk-Taking  



Organization & Coherence  



Use of Sources/Citation  



Tone/Style/Grammar  
